In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import activations, layers, models, losses, optimizers, datasets, utils

In [2]:
from functools import reduce
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

In [3]:
def ResCBN(
    filters,kernel_size=3,
    activation='relu',strides=1,
    padding="valid"):
    
    conv=layers.Conv2D(
        filters, kernel_size=kernel_size,
        strides=strides,padding=padding)
    bn=layers.BatchNormalization(epsilon=1.001e-05)
    layers_list=[conv,bn]
    if activation:
        act=layers.Activation(activation=activation)
        layers_list.append(act)
    return compose(*layers_list)

def ResBlockV1(x,filters,kernel_sizes=[1,3,1,1],strides=1):
    x1=compose(
        ResCBN(filters[0],kernel_size=kernel_sizes[0],strides=strides),
        ResCBN(filters[1],kernel_size=kernel_sizes[1],padding="same"),
        ResCBN(filters[2],kernel_size=kernel_sizes[2],activation=None)
    )(x)
    x2=ResCBN(
        filters[3],kernel_size=kernel_sizes[3],
        strides=strides,
        activation=None)(x)
    x=layers.Add()([x2,x1])
    x=layers.Activation(activations.relu)(x)
    return x
    
def ResBlockV2(x,filters,kernel_sizes=[1,3,1]):
    x1=compose(
        ResCBN(filters[0],kernel_size=kernel_sizes[0]),
        ResCBN(filters[1],kernel_size=kernel_sizes[1],padding="same"),
        ResCBN(filters[2],kernel_size=kernel_sizes[2],activation=None)
    )(x)
    x=layers.Add()([x1,x])
    x=layers.Activation(activations.relu)(x)
    return x

In [4]:
inp=layers.Input(shape=(224,224,3))
x=compose(
    layers.ZeroPadding2D(padding=3),
    layers.Conv2D(64, 7, strides=2),
    layers.BatchNormalization(epsilon=1.001e-05),
    layers.Activation(activations.relu),
    layers.ZeroPadding2D(),
    layers.MaxPooling2D(pool_size=3,strides=2)
)(inp)
x=ResBlockV1(x,filters=[64,64,256,256])
x=ResBlockV2(x,filters=[64,64,256])
x=ResBlockV2(x,filters=[64,64,256])

x=ResBlockV1(x,filters=[128,128,512,512],strides=2)
x=ResBlockV2(x,filters=[128,128,512])
x=ResBlockV2(x,filters=[128,128,512])
x=ResBlockV2(x,filters=[128,128,512])

x=ResBlockV1(x,filters=[256,256,1024,1024],strides=2)
x=ResBlockV2(x,filters=[256,256,1024])
x=ResBlockV2(x,filters=[256,256,1024])
x=ResBlockV2(x,filters=[256,256,1024])
x=ResBlockV2(x,filters=[256,256,1024])
x=ResBlockV2(x,filters=[256,256,1024])

x=ResBlockV1(x,filters=[512,512,2048,2048],strides=2)
x=ResBlockV2(x,filters=[512,512,2048])
x=ResBlockV2(x,filters=[512,512,2048])
x=layers.GlobalAveragePooling2D()(x)
out=layers.Dense(1000,activation="softmax")(x)
copy_model=models.Model(inp,out)

2024-04-17 10:22:28.085504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:22:28.085760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:22:28.091451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:22:28.091725: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:22:28.091923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [5]:
origin_model=tf.keras.applications.ResNet50(
    input_shape=(224,224,3),include_top=True)
copy_model.set_weights(origin_model.get_weights())

for org, new in zip(origin_model.layers, copy_model.layers):
    new._name=org.name